In [16]:
import json
from pathlib import Path
import os

def read_image_captions(folder_path):
    image_captions = {}

    for file in Path(folder_path).glob("*.jsonl"):
        file_image_captions = read_jsonl_to_list(file)
        file_base, extension = os.path.splitext(os.path.basename(file))
        image_captions[file_base] = file_image_captions
    return image_captions

def read_jsonl_to_list(file_path):
    result = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                json_obj = json.loads(line)
                for key, value in json_obj.items():
                    result.append([key.split('.')[0], value])
            except json.JSONDecodeError:
                print(f"Skipping invalid JSON line: {line}")
    return result


image_captions = read_image_captions("data/image-caption")

In [18]:
image_captions["L13_V001"]

[['001',
  'Bức ảnh chụp một thành phố với những tòa nhà cao tầng được chiếu sáng rực rỡ. Phía trước là một dòng sông uốn lượn, tạo nên một khung cảnh đẹp mắt. Ở trung tâm bức ảnh là logo "giây" được thiết kế theo phong cách hiện đại, nổi bật trên nền trời hoàng hôn. Phía trên cùng bên phải của bức ảnh là logo "HTV9" và dòng chữ "06:30:22 HD".'],
 ['002',
  'Bức ảnh chụp một studio truyền hình, có hai người đang đứng trên bục phát thanh. Người đàn ông mặc áo sơ mi trắng, đeo cà vạt xanh, tay cầm micro. Người phụ nữ mặc áo sơ mi trắng, đeo cà vạt xanh, tay cầm micro. Phía sau họ là một bức tranh phong cảnh thành phố với những tòa nhà cao tầng và bầu trời hoàng hôn. Bức tranh được thiết kế theo phong cách hiện đại, với màu sắc tươi sáng. Ở góc trên bên phải của bức ảnh, có dòng chữ "HTV9" và "06:50:26 HD".'],
 ['003',
  'Bức ảnh chụp một studio truyền hình với hai người đang đứng trước máy quay. Người đàn ông mặc áo sơ mi trắng, đeo cà vạt xanh, tay cầm điện thoại. Người phụ nữ mặc áo sơ

In [21]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

In [24]:
text_input = "Một con thuyền chạy được trên băng, màu đen. Con thuyền này chạy bằng động cơ cánh quạt ở bên trên thổi hướng ra phía sau. Con thuyền là phương tiện hỗ trợ cứu hộ một nạn nhân bị rơi xuống hồ băng."
inputs_sample = tokenizer(text_input, return_tensors="pt", padding=True, truncation=True)
outputs_sample = model(**inputs_sample)
embeddings_sample = outputs_sample.last_hidden_state.mean(dim=1).detach().numpy()


max = None
max_score = 0 

for caption in image_captions["L13_V001"]:
    inputs_candidate = tokenizer(caption, return_tensors="pt", padding=True, truncation=True)
    outputs_candidate = model(**inputs_candidate)
    embeddings_candidate = outputs_candidate.last_hidden_state.mean(dim=1).detach().numpy()
    similarity = np.dot(embeddings_sample, embeddings_candidate.T) / (np.linalg.norm(embeddings_sample) * np.linalg.norm(embeddings_candidate))
    
    if (similarity[0][0] > max_score):
        max_score = similarity[0][0]
        max = caption
print(max)
print(max_score)